# Getting started with GitHub Models - OpenAI SDK

## 1. Personal access token

A personal access token is made available in the Codespaces environment in the `GITHUB_TOKEN` environment variable.

## 2. Install dependencies

In [ ]:
%pip install openai --quiet
%pip install python-dotenv --quiet



## 3. Set environment variables and create the client




In [ ]:
import os
import dotenv
from openai import OpenAI

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.github.ai/inference"


model_name = "openai/gpt-4o-mini" # or "openai/gpt-4o"
client = OpenAI()

## 4. Run a basic code sample

This is just calling the `chat.completions` endpoint with a simple prompt.




In [ ]:

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of France?",
        }
    ],
    model=model_name,
    # Optional parameters
    temperature=1.,
    max_tokens=1000,
    top_p=1.    
)

print(response.choices[0].message.content)

## 5. Multi-Turn Conversation

This sample demonstrates a multi-turn conversation with the chat completion API.
When using the model for a chat application, you'll need to manage the history of that
conversation and send the latest messages to the model.



In [ ]:
# Call the chat completion API
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of France?",
        },
        {
            "role": "assistant",
            "content": "The capital of France is Paris.",
        },
        {
            "role": "user",
            "content": "What about Spain?",
        },
    ],
    model=model_name,
)

# Print the response
print(response.choices[0].message.content)


## 6. Using images as inputs

The `gpt-4o-mini` model supports using images as inputs. To run a chat completion using
a local image file, use the following sample. To send it to the service, you'll need to encode the image as **data URI**, which is a string that starts with `data:image/png;base64,` followed by the base64-encoded image.
We are using this small image as an example: 

![image](./sample.png)



In [ ]:
import base64
model_name = "openai/gpt-4o-mini"


def get_image_data_url(image_file: str, image_format: str) -> str:
    """
    Helper function to converts an image file to a data URL string.

    Args:
        image_file (str): The path to the image file.
        image_format (str): The format of the image file.

    Returns:
        str: The data URL of the image.
    """
    try:
        with open(image_file, "rb") as f:
            image_data = base64.b64encode(f.read()).decode("utf-8")
    except FileNotFoundError:
        print(f"Could not read '{image_file}'.")
        exit()
    return f"data:image/{image_format};base64,{image_data}"


# Call the chat completion API
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that describes images in detail.",
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        # using a file located in this directory
                        "url": get_image_data_url("./sample.png", "png")
                    },
                },
            ],
        },
    ],
    model=model_name,
)

# Print the response
print(response.choices[0].message.content)


## 7. Streaming the response

For a better user experience, you will want to stream the response of the model
so that the first token shows up early and you avoid waiting for long responses.






In [ ]:
# Call the chat completion API
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Give me 5 good reasons why I should exercise every day.",
        },
    ],
    model=model_name,
    stream=True,
)

# Print the streamed response
for update in response:
    if update.choices[0].delta.content:
        print(update.choices[0].delta.content, end="")


## 8. Tools and Function Calling

A language model like `gpt-4o-mini` can be given a set of tools it can ask the calling program to invoke,
for running specific actions depending on the context of the conversation.
This sample demonstrates how to define a function tool and how to act on a request from the model to invoke it.


In [ ]:
import json

# Define a function that returns flight information between two cities (mock implementation)
def get_flight_info(origin_city: str, destination_city: str):
    if origin_city == "Seattle" and destination_city == "Miami":
        return json.dumps(
            {
                "airline": "Delta",
                "flight_number": "DL123",
                "flight_date": "May 7th, 2024",
                "flight_time": "10:00AM",
            }
        )
    return json.dump({"error": "No flights found between the cities"})


# Define a function tool that the model can ask to invoke in order to retrieve flight information
tool = {
    "type": "function",
    "function": {
        "name": "get_flight_info",
        "description": """Returns information about the next flight between two cities.
            This includes the name of the airline, flight number and the date and time
            of the next flight""",
        "parameters": {
            "type": "object",
            "properties": {
                "origin_city": {
                    "type": "string",
                    "description": "The name of the city where the flight originates",
                },
                "destination_city": {
                    "type": "string",
                    "description": "The flight destination city",
                },
            },
            "required": ["origin_city", "destination_city"],
        },
    },
}


messages = [
    {
        "role": "system",
        "content": "You an assistant that helps users find flight information.",
    },
    {
        "role": "user",
        "content": "I'm interested in going to Miami. What is the next flight there from Seattle?",
    },
]

response = client.chat.completions.create(
    messages=messages,
    tools=[tool],
    model=model_name,
)

# We expect the model to ask for a tool call
if response.choices[0].finish_reason == "tool_calls":

    # Append the model response to the chat history
    messages.append(response.choices[0].message)

    # We expect a single tool call
    if (
        response.choices[0].message.tool_calls
        and len(response.choices[0].message.tool_calls) == 1
    ):

        tool_call = response.choices[0].message.tool_calls[0]

        # We expect the tool to be a function call
        if tool_call.type == "function":

            # Parse the function call arguments and call the function
            function_args = json.loads(tool_call.function.arguments.replace("'", '"'))
            print(
                f"Calling function `{tool_call.function.name}` with arguments {function_args}"
            )
            callable_func = locals()[tool_call.function.name]
            function_return = callable_func(**function_args)
            print(f"Function returned = {function_return}")

            # Append the function call result fo the chat history
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": function_return,
                }
            )

            # Get another response from the model
            response = client.chat.completions.create(
                messages=messages,
                tools=[tool],
                model=model_name,
            )

            print(f"Model response = {response.choices[0].message.content}")

## Next Steps

To learn more about what you can do with the GitHub models using Python, check out the following cookbooks:

- [How to process image and video with GPT-4](../../../cookbooks/python/openai/how_to_process_image_and_video_with_gpt4o.ipynb): This notebook shows how to process images and videos with GPT-4.
- [How to call functions with chat models](../../../cookbooks/python/openai/How_to_call_functions_with_chat_models.ipynb): This notebook shows how to get GPT-4o to determing which of a set of functions to call to answer a user's question.
- [RAG getting started](../../../cookbooks/python/llamaindex/rag_getting_started.ipynb):
This Jupyter Notebook demonstrates the use of Retrieval-Augmented Generation (RAG) with LLamaIndex to create a question-answering system that retrieves relevant information from a collection of documents and generates contextually appropriate responses.

Or check out [this folder for more cookbooks](../../../cookbooks/python/README.md).
